In [36]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from sklearn.metrics import classification_report,confusion_matrix
import ipywidgets as widgets
import io
from sklearn.metrics import accuracy_score
from PIL import Image
from IPython.display import display,clear_output
from warnings import filterwarnings

In [2]:
labels = ['glioma_tumor','no_tumor','meningioma_tumor','pituitary_tumor']

In [3]:
X_train = []
y_train = []
image_size = 150
for i in labels:
    folderPath = os.path.join('Training',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size, image_size))
        X_train.append(img)
        y_train.append(i)
      
    
for i in labels:
    folderPath = os.path.join('Testing',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_train.append(img)
        y_train.append(i)
        

100%|██████████| 74/74 [00:01<00:00, 63.58it/s]


In [4]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [5]:
y_train


array(['glioma_tumor', 'glioma_tumor', 'glioma_tumor', ...,
       'pituitary_tumor', 'pituitary_tumor', 'pituitary_tumor'],
      dtype='<U16')

In [6]:
X_train, y_train = shuffle(X_train,y_train, random_state=101)


In [7]:
X_train.shape

(3264, 150, 150, 3)

# Split dataset int train and test sets

In [8]:
X_train,X_test,y_train,y_test = train_test_split(X_train,y_train, test_size=0.1,random_state=101)


# As multiclass classification, so i had perform OneHotEncoding

In [9]:
y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)


y_test_new = []
for i in y_test:
    y_test_new.append(labels.index(i))
y_test = y_test_new
y_test = tf.keras.utils.to_categorical(y_test)

# Transfer Learning:
Transfer learning is a machine learning method where we reuse a pre-trained model as the starting point for a model on a new task. To put it simply—a model trained on one task is repurposed on a second, related task as an optimization that allows rapid progress when modeling the second task

In this notebook, I'll be using the VGG16 and EfficientNetB0 model which will use the weights from the ImageNet dataset.

The include_top parameter is set to False so that the network doesn't include the top layer/ output layer from the pre-built model which allows us to add our own output layer depending upon our use case!

In [10]:
base_model = VGG16(input_shape = (150, 150, 3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

In [11]:
base_model.trainable=False


# Adding top_layers

In [12]:
model1 = base_model.output

model1 = tf.keras.layers.Dense(512,activation='relu')(model1)
model1 = tf.keras.layers.GlobalAveragePooling2D()(model1)
model1 = tf.keras.layers.Dropout(rate=0.5)(model1)
model1 = tf.keras.layers.Dense(4,activation='softmax')(model1)
model1 = tf.keras.models.Model(inputs=base_model.input, outputs = model1)

In [13]:
model1.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

# Callback preperation

A callback is an object that can perform actions at various stages of training (e.g. at the start or end of an epoch, before or after a single batch, etc). You can use callbacks to: Write TensorBoard logs after every batch of training to monitor your metrics. Periodically save your model to disk.

In [14]:
tensorboard1 = TensorBoard(log_dir = 'logs')
checkpoint1 = ModelCheckpoint("vgg16",monitor="val_accuracy",save_best_only=True,mode="auto",verbose=1)
reduce_lr1 = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.4, patience = 2, min_delta = 0.0001,
                              mode='auto',verbose=1)

# Compile our model first model


In [15]:
model1.compile(optimizer="Adam",loss='categorical_crossentropy', metrics=['accuracy'])


# Training Process of Model



In [16]:
history1 = model1.fit(X_train,y_train,validation_split=0.1, epochs =12, verbose=1, batch_size=32,
                      callbacks=[tensorboard1,checkpoint1,reduce_lr1])


Epoch 1/12
83/83 [==============================] - ETA: 0s - loss: 2.8885 - accuracy: 0.6220
Epoch 1: val_accuracy improved from -inf to 0.76871, saving model to vgg16


INFO:tensorflow:Assets written to: vgg16\assets


INFO:tensorflow:Assets written to: vgg16\assets


83/83 [==============================] - 241s 3s/step - loss: 2.8885 - accuracy: 0.6220 - val_loss: 0.8622 - val_accuracy: 0.7687 - lr: 0.0010
Epoch 2/12
83/83 [==============================] - ETA: 0s - loss: 0.8839 - accuracy: 0.7719
Epoch 2: val_accuracy improved from 0.76871 to 0.80952, saving model to vgg16


INFO:tensorflow:Assets written to: vgg16\assets


...

INFO:tensorflow:Assets written to: vgg16\assets


83/83 [==============================] - 242s 3s/step - loss: 0.1189 - accuracy: 0.9633 - val_loss: 0.3833 - val_accuracy: 0.8844 - lr: 4.0000e-04
Epoch 10/12
83/83 [==============================] - ETA: 0s - loss: 0.1128 - accuracy: 0.9629
Epoch 10: val_accuracy improved from 0.88435 to 0.88776, saving model to vgg16


INFO:tensorflow:Assets written to: vgg16\assets


INFO:tensorflow:Assets written to: vgg16\assets


83/83 [==============================] - 248s 3s/step - loss: 0.1128 - accuracy: 0.9629 - val_loss: 0.3866 - val_accuracy: 0.8878 - lr: 4.0000e-04
Epoch 11/12
83/83 [==============================] - ETA: 0s - loss: 0.0927 - accuracy: 0.9765
Epoch 11: val_accuracy did not improve from 0.88776
83/83 [==============================] - 253s 3s/step - loss: 0.0927 - accuracy: 0.9765 - val_loss: 0.4041 - val_accuracy: 0.8878 - lr: 4.0000e-04
Epoch 12/12
83/83 [==============================] - ETA: 0s - loss: 0.0949 - accuracy: 0.9678
Epoch 12: val_accuracy did not improve from 0.88776

Epoch 12: ReduceLROnPlateau reducing learning rate to 0.00016000000759959222.
83/83 [==============================] - 248s 3s/step - loss: 0.0949 - accuracy: 0.9678 - val_loss: 0.3916 - val_accuracy: 0.8878 - lr: 4.0000e-04


In [20]:
model1.save("brain_model.h5")

In [24]:
pred = model1.predict(X_test)
pred = np.argmax(pred,axis=1)
y_test_new = np.argmax(y_test,axis=1)

11/11 [==============================] - 27s 2s/step


In [25]:
prediction = pd.DataFrame(pred, columns=['predictions']).to_csv('prediction.csv')

# Using another pretrained model EfficientNetB0!

In [26]:
effnet = EfficientNetB0(weights='imagenet',include_top=False,input_shape=(image_size,image_size,3))

In [27]:
model2 = effnet.output
model2 = tf.keras.layers.GlobalAveragePooling2D()(model2)
model2 = tf.keras.layers.Dropout(rate=0.5)(model2)
model2 = tf.keras.layers.Dense(4,activation='softmax')(model2)
model2 = tf.keras.models.Model(inputs=effnet.input, outputs = model2)

In [28]:
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 150, 150, 3)  0           ['input_3[0][0]']                
                                                                                                  
 normalization_1 (Normalization  (None, 150, 150, 3)  7          ['rescaling_1[0][0]']            
 )                                                                                                
                                                                                            

                                                                                                  
 block2b_expand_conv (Conv2D)   (None, 38, 38, 144)  3456        ['block2a_project_bn[0][0]']     
                                                                                                  
 block2b_expand_bn (BatchNormal  (None, 38, 38, 144)  576        ['block2b_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block2b_expand_activation (Act  (None, 38, 38, 144)  0          ['block2b_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block2b_dwconv (DepthwiseConv2  (None, 38, 38, 144)  1296       ['block2b_expand_activation[0][0]
 D)       

 ivation)                                                                                         
                                                                                                  
 block3b_dwconv (DepthwiseConv2  (None, 19, 19, 240)  6000       ['block3b_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block3b_bn (BatchNormalization  (None, 19, 19, 240)  960        ['block3b_dwconv[0][0]']         
 )                                                                                                
                                                                                                  
 block3b_activation (Activation  (None, 19, 19, 240)  0          ['block3b_bn[0][0]']             
 )                                                                                                
          

                                                                                                  
 block4b_activation (Activation  (None, 10, 10, 480)  0          ['block4b_bn[0][0]']             
 )                                                                                                
                                                                                                  
 block4b_se_squeeze (GlobalAver  (None, 480)         0           ['block4b_activation[0][0]']     
 agePooling2D)                                                                                    
                                                                                                  
 block4b_se_reshape (Reshape)   (None, 1, 1, 480)    0           ['block4b_se_squeeze[0][0]']     
                                                                                                  
 block4b_se_reduce (Conv2D)     (None, 1, 1, 20)     9620        ['block4b_se_reshape[0][0]']     
          

 agePooling2D)                                                                                    
                                                                                                  
 block5a_se_reshape (Reshape)   (None, 1, 1, 480)    0           ['block5a_se_squeeze[0][0]']     
                                                                                                  
 block5a_se_reduce (Conv2D)     (None, 1, 1, 20)     9620        ['block5a_se_reshape[0][0]']     
                                                                                                  
 block5a_se_expand (Conv2D)     (None, 1, 1, 480)    10080       ['block5a_se_reduce[0][0]']      
                                                                                                  
 block5a_se_excite (Multiply)   (None, 10, 10, 480)  0           ['block5a_activation[0][0]',     
                                                                  'block5a_se_expand[0][0]']      
          

                                                                                                  
 block5c_project_conv (Conv2D)  (None, 10, 10, 112)  75264       ['block5c_se_excite[0][0]']      
                                                                                                  
 block5c_project_bn (BatchNorma  (None, 10, 10, 112)  448        ['block5c_project_conv[0][0]']   
 lization)                                                                                        
                                                                                                  
 block5c_drop (Dropout)         (None, 10, 10, 112)  0           ['block5c_project_bn[0][0]']     
                                                                                                  
 block5c_add (Add)              (None, 10, 10, 112)  0           ['block5c_drop[0][0]',           
                                                                  'block5b_add[0][0]']            
          

                                                                                                  
 block6b_add (Add)              (None, 5, 5, 192)    0           ['block6b_drop[0][0]',           
                                                                  'block6a_project_bn[0][0]']     
                                                                                                  
 block6c_expand_conv (Conv2D)   (None, 5, 5, 1152)   221184      ['block6b_add[0][0]']            
                                                                                                  
 block6c_expand_bn (BatchNormal  (None, 5, 5, 1152)  4608        ['block6c_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block6c_expand_activation (Act  (None, 5, 5, 1152)  0           ['block6c_expand_bn[0][0]']      
 ivation) 

                                                                                                  
 block7a_expand_bn (BatchNormal  (None, 5, 5, 1152)  4608        ['block7a_expand_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 block7a_expand_activation (Act  (None, 5, 5, 1152)  0           ['block7a_expand_bn[0][0]']      
 ivation)                                                                                         
                                                                                                  
 block7a_dwconv (DepthwiseConv2  (None, 5, 5, 1152)  10368       ['block7a_expand_activation[0][0]
 D)                                                              ']                               
                                                                                                  
 block7a_b

In [29]:
tensorboard2 = TensorBoard(log_dir = 'logs')
checkpoint2 = ModelCheckpoint("effnet.h5",monitor="val_accuracy",save_best_only=True,mode="auto",verbose=1)
reduce_lr2 = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.3, patience = 2, min_delta = 0.0001,
                              mode='auto',verbose=1)

In [30]:
model2.compile(optimizer="Adam",loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
history = model2.fit(X_train,y_train,validation_split=0.1, epochs =10, verbose=1, batch_size=32,
                   callbacks=[tensorboard2,checkpoint2,reduce_lr2])

Epoch 1/10
83/83 [==============================] - ETA: 0s - loss: 0.4553 - accuracy: 0.8207
Epoch 1: val_accuracy improved from -inf to 0.91156, saving model to effnet.h5
83/83 [==============================] - 281s 3s/step - loss: 0.4553 - accuracy: 0.8207 - val_loss: 0.3393 - val_accuracy: 0.9116 - lr: 0.0010
Epoch 2/10
83/83 [==============================] - ETA: 0s - loss: 0.1970 - accuracy: 0.9379
Epoch 2: val_accuracy did not improve from 0.91156
83/83 [==============================] - 255s 3s/step - loss: 0.1970 - accuracy: 0.9379 - val_loss: 0.5336 - val_accuracy: 0.8435 - lr: 0.0010
Epoch 3/10
83/83 [==============================] - ETA: 0s - loss: 0.1289 - accuracy: 0.9546
Epoch 3: val_accuracy improved from 0.91156 to 0.91837, saving model to effnet.h5
83/83 [==============================] - 256s 3s/step - loss: 0.1289 - accuracy: 0.9546 - val_loss: 0.2416 - val_accuracy: 0.9184 - lr: 0.0010
Epoch 4/10
83/83 [==============================] - ETA: 0s - loss: 0.0964 - 

In [32]:
model2.save("brain_model.h5")

In [33]:
pred1 = model1.predict(X_test)
pred1 = np.argmax(pred,axis=0)
y_test_new1 = np.argmax(y_test,axis=1)

11/11 [==============================] - 16s 1s/step


In [34]:
pred2 = model2.predict(X_test)
pred2 = np.argmax(pred,axis=0)
y_test_new2 = np.argmax(y_test,axis=1)

11/11 [==============================] - 7s 526ms/step
